In [1]:
import scanpy as sc
import numpy as np
import os
import anndata2ri
import pathlib
from scipy import io
import anndata#
import pandas as pd
from tqdm import tqdm
import argparse
import sys

# Activate the anndata2ri conversion between SingleCellExperiment and AnnData
# anndata2ri.activate()

#Loading the rpy2 extension enables cell magic to be used
#This runs R code in jupyter notebook cells
# %load_ext rpy2.ipython

sc.settings.verbosity = 3
# sc.logging.print_versions()

import warnings
warnings.filterwarnings("ignore")
    
object_name = "integrated_BrainMet_dataset.output.s8_harmony.cluster.0.5"
path_to_seurat2anndata = "/media/hieunguyen/GSHD_HN01/outdir/BrainMet_SeuratV5/integrate_BrainMet_datasets/integrated_v0.2/seurat2anndata/from_12_output/integrated_BrainMet_dataset.output.s8"
os.makedirs(path_to_seurat2anndata, exist_ok=True)

##### MAIN CONVERSION FUNCTIONS
print(os.path.join(path_to_seurat2anndata, "counts_{}.mtx".format(object_name)))
X = io.mmread(os.path.join(path_to_seurat2anndata, "counts_{}.mtx".format(object_name)))

# create anndata object
adata = anndata.AnnData(X=X.transpose().tocsr())

# load cell metadata:
cell_meta = pd.read_csv(os.path.join(path_to_seurat2anndata, "metadata_{}.csv".format(object_name)))

# load gene names:
with open(os.path.join(path_to_seurat2anndata, "gene_names_{}.csv".format(object_name)), 'r') as f:
    gene_names = f.read().splitlines()

# set anndata observations and index obs by barcodes, var by gene names
adata.obs = cell_meta
adata.obs.index = adata.obs['barcode']
adata.var.index = gene_names

# load dimensional reduction:
pca = pd.read_csv(os.path.join(path_to_seurat2anndata, "pca_{}.csv".format(object_name)))
pca.index = adata.obs.index

# set pca and umap
adata.obsm['X_pca'] = pca.to_numpy()
adata.obsm['X_umap'] = np.vstack((adata.obs['UMAP_1'].to_numpy(), adata.obs['UMAP_2'].to_numpy())).T

# save dataset as anndata format
adata.write(os.path.join(path_to_seurat2anndata, '{}.h5ad'.format(object_name)))
        

/media/hieunguyen/GSHD_HN01/outdir/BrainMet_SeuratV5/integrate_BrainMet_datasets/integrated_v0.2/seurat2anndata/from_12_output/integrated_BrainMet_dataset.output.s8/counts_integrated_BrainMet_dataset.output.s8_harmony.cluster.0.5.mtx
